<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Template para o Colab do Projeto Semestral**
---

Atenção, podem ser que nem todas as tarefas sejam executadas no Colab (a aplicação por exemplo, pode estar hospedada no streamlit cloud). Mas a maior parte pode estar aqui ou ao menos indicada e comentada.


Além disso a entrega deve incluir:

1. **Um GitHub público do projeto**
2. **Código completo e executável em um notebook Python (este template)**
3. **Uma aplicação streamlit para consumo do modelo**
4. **Um texto/artigo do projeto**
5. **Um vídeo (link YouTube ou outro) de no máximo 3min de apresentação do projeto**

Um **`readme.md`** no GitHub público do projeto deve indicar (um índice) cada uma dessas entregas.








In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '10401436, Rafael de Souza Oliveira Cerqueira Tinôco' #@param {type:"string"}
Aluno2 = 'None' #@param {type:"string"}
Aluno3 = 'None' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}



In [ ]:
#@title Assinale aqui a sua opção de Projeto
Projeto = "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos" #@param ["IA Aplicada a Imagens: Uso de Modelos de Redes Neurais", "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos"]




# **Resumo**

## 1. Objetivo do Projeto
Desenvolver uma solução de IA para processamento e classificação automática de efeitos adversos em bulas de medicamentos, com capacidade para:

- Extrair e estruturar informações técnicas de arquivos PDF
- Identificar e categorizar sintomas com precisão clínica
- Classificar a gravidade dos efeitos adversos (leve/grave)
- Gerar relatórios quantitativos para análise farmacológica

## 2. Fonte dos Dados

Os dados foram coletados através do site: https://consultaremedios.com.br/. Foram pesquisados remédios na seção de TDAH, anti-depressivos e ansiedade. Assim que um remédio era achado foi procurado as bulas originais dos remédios, que estão disponíveis para download.

## 3. Arquitetura Técnica

### Processamento de Dados
- **Processamento de PDF** é feito com PyPDF2 e PyCryptodome para extração robusta de texto, incluindo arquivos criptografados
- **Pré-processamento de texto** utiliza expressões regulares (regex) e tabelas de sinônimos para normalização terminológica
- **Identificação de seções** é realizada através de algoritmos de pattern matching em texto não-estruturado

### Modelos de IA
- **BERTimbau** (versão em português do BERT) para análise semântica e classificação contextual
- **Sentence-Transformers** para embedding de textos médicos
- **Scikit-learn** (OPCIONAL) para modelos auxiliares de validação

### Visualização
- **Streamlit** para construção da interface web interativa
- **Plotly/Matplotlib** para geração de visualizações dinâmicas
- **Pandas** para manipulação e exportação de dados tabulares

## 3. Resultados Esperados

### Saída Quantitativa
{
  "Arquivo": "bula_paracetamol.pdf",
  "Total_Sintomas": 42,
  "Leves": 35 (83%),
  "Graves": 7 (17%),
  "Sistemas_Afetados": ["hepático", "gastrointestinal"]
}

# **Apresentação dos dados**

https://uploads.consultaremedios.com.br/drug_leaflet/pro/Bula-Venvanse-Profissional-Consulta-Remedios.pdf

# Abaixo algumas seções de exemplo

> Pode haver mais, dependendo da sua aplicação. Para cada seção faça comentários explicando a tarefa e comentando/sumarizando os resultados.

# **Instalar dependências**

In [ ]:
!pip install PyPDF2 transformers torch pandas matplotlib PyCryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.2 MB/s eta 0:00:00


# **Preparação e transformação dos dados**



In [ ]:
import os
from PyPDF2 import PdfReader
from PyPDF2.errors import DependencyError

TERMOS_MEDICOS = {
    "sintomas": [
        "dor", "febre", "náusea", "vômito", "tontura", "cefaleia", "confusão",
        "fraqueza", "paralisia", "dificuldade de falar", "batimento cardíaco acelerado",
        "dor no peito", "equimose", "espasmos", "tiques", "garganta inflamada",
        "convulsões", "coceiras", "manchas vermelhas", "dificuldade de respirar",
        "erupção cutânea", "urticária", "transpiração", "dor no estômago",
        "desânimo", "cansaço", "cãibra", "boca seca", "visão borrada",
        "perda de peso", "perda de cabelo", "alucinações", "movimentos bruscos",
        "movimentos contorcidos", "degustação", "bolhas na pele", "prurido",
        "rash", "fadiga", "edema", "hemorragia", "icterícia", "parestesia",
        "vertigem", "síncope", "dispneia", "palpitação", "taquicardia",
        "hipertensão", "hipotensão", "diarreia", "constipação", "flatulência",
        "pirose", "dispepsia", "anorexia", "astenia", "mialgia", "artralgia",
        "rigidez", "tremor", "poliúria", "oligúria", "anúria", "disúria",
        "hematúria", "proteinúria", "tosse", "expectoração", "rinorreia",
        "epistaxe", "otalgia", "tinnitus", "fotofobia", "diplopia", "xerostomia",
        "gengivorragia", "glossodinia", "linfadenopatia", "hepatomegalia",
        "esplenomegalia", "ascite", "cianose", "pallor", "eritema", "petéquias",
        "púrpura", "necrose", "úlcera", "fissura", "descamação", "hiperqueratose"
    ],
    "sistemas": [
        "gastrointestinal", "respiratório", "cardiovascular", "neurológico",
        "dermatológico", "hematológico", "hepático", "renal", "endócrino",
        "musculoesquelético", "imunológico", "oftalmológico", "otorrinolaringológico",
        "geniturinário", "psiquiátrico"
    ],
    "frequencia": {
        "muito_comum": ["muito comum", "≥10%", "10%", "muito frequente"],
        "comum": ["comum", "≥1%", "1%", "frequente"],
        "incomum": ["incomum", "≥0.1%", "0.1%", "pouco frequente"],
        "raro": ["raro", "≤0.01%", "0.01%", "muito raro"]
    }
}

SINONIMOS = {
    "dor de cabeça": "cefaleia",
    "enjoo": "náusea",
    "coceira": "prurido",
    "mancha na pele": "rash",
    "cansaço": "fadiga",
    "tonturas": "vertigem",
    "desmaio": "síncope",
    "falta de ar": "dispneia",
    "azia": "pirose",
    "indigestão": "dispepsia",
    "sangramento": "hemorragia",
    "amarelecimento": "icterícia",
    "formigamento": "parestesia"
}

CATEGORIAS = {
    'leve': TERMOS_MEDICOS["frequencia"]["muito_comum"] +
            TERMOS_MEDICOS["frequencia"]["comum"] +
            ["leve", "suave", "moderado", "transitório", "temporário", "benigno"],
    'grave': TERMOS_MEDICOS["frequencia"]["incomum"] +
             TERMOS_MEDICOS["frequencia"]["raro"] +
             ["grave", "severa", "severos", "reação", "reações", "adversa",
              "anafilaxia", "choque", "hospitalização", "emergência", "morte",
              "fatal", "irreversível", "crônico", "progressivo", "maligno",
              "toxicidade", "overdose", "intoxicação", "insuficiência"]
}

def extrair_texto(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)

            if reader.is_encrypted:
                try:
                    reader.decrypt('')
                except Exception:
                    print(f"PDF protegido não pôde ser lido: {os.path.basename(pdf_path)}")
                    return None

            texto = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    texto += page_text + "\n"
            return texto if texto.strip() else None

    except DependencyError:
        print(f"Erro ao ler {os.path.basename(pdf_path)}: Instale PyCryptodome com 'pip install pycryptodome'")
        return None
    except Exception as e:
        print(f"Erro ao ler {os.path.basename(pdf_path)}: {str(e)}")
        return None

def normalizar_termos(texto):
    if not texto:
        return ""
    texto = texto.lower()
    for termo, sinonimo in SINONIMOS.items():
        texto = texto.replace(termo, sinonimo)
    return texto

def separar_secoes(texto):
    texto = normalizar_termos(texto)
    secoes_relevantes = {
        'Reações Adversas': '',
        'Efeitos Colaterais': '',
        'Advertências': '',
        'Precauções': '',
        'Interações': '',
        'Contraindicações': ''
    }

    secao_atual = None
    linhas = texto.split('\n')

    for linha in linhas:
        linha_limpa = linha.strip().lower()

        if 'reações adversas' in linha_limpa:
            secao_atual = 'Reações Adversas'
        elif 'efeitos colaterais' in linha_limpa:
            secao_atual = 'Efeitos Colaterais'
        elif 'advertências' in linha_limpa:
            secao_atual = 'Advertências'
        elif 'precauções' in linha_limpa:
            secao_atual = 'Precauções'
        elif 'interações' in linha_limpa:
            secao_atual = 'Interações'
        elif 'contraindicações' in linha_limpa:
            secao_atual = 'Contraindicações'
        elif secao_atual and linha:
            secoes_relevantes[secao_atual] += linha + " "
        else:
            secao_atual = None

    return secoes_relevantes

# **Avaliação do modelo**



In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def carregar_modelo():
    MODEL_NAME = 'neuralmind/bert-base-portuguese-cased'
    tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
    model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
    return tokenizer, model

def classificar_frase(frase, tokenizer, model):
    inputs = tokenizer(
        frase,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding=True
    )

    with torch.no_grad():
        outputs = model(**inputs)

    predicao = torch.argmax(outputs.logits, dim=1).item()
    return 'leve' if predicao == 0 else 'grave'

# **Consumo do modelo**

In [ ]:
import os
import streamlit as st
import pandas as pd
from preparacao_dados import extrair_texto, normalizar_termos, separar_secoes, TERMOS_MEDICOS, CATEGORIAS
from modelo import carregar_modelo, classificar_frase

# Configurações
PASTA_EXEMPLOS = 'exemplos'

def processar_bula(texto, tokenizer, model, pdf_path):
    """Processa o texto da bula e classifica os sintomas"""
    if not texto:
        return pd.DataFrame()

    secoes_relevantes = separar_secoes(texto)
    resultados = []

    for secao, conteudo in secoes_relevantes.items():
        if not conteudo:
            continue

        frases = [f.strip() for f in conteudo.split('.') if f.strip()]

        for frase in frases:
            frase_lower = frase.lower()

            # Verificar se há termos médicos na frase
            termos_encontrados = [
                termo for termo in TERMOS_MEDICOS["sintomas"]
                if termo in frase_lower
            ]

            if termos_encontrados:
                try:
                    # Classificação com BERT
                    classificacao = classificar_frase(frase, tokenizer, model)

                    # Refinamento com palavras-chave
                    if any(palavra in frase_lower for palavra in CATEGORIAS['grave']):
                        classificacao = 'grave'
                    elif any(palavra in frase_lower for palavra in CATEGORIAS['leve']):
                        classificacao = 'leve'

                    # Identificar sistemas afetados
                    sistemas_afetados = [
                        s for s in TERMOS_MEDICOS["sistemas"]
                        if s in frase_lower
                    ]

                    resultados.append({
                        'Arquivo': os.path.basename(pdf_path),
                        'Seção': secao,
                        'Termos Encontrados': ', '.join(termos_encontrados),
                        'Texto': frase,
                        'Classificação': classificacao,
                        'Sistema Afetado': ', '.join(sistemas_afetados)
                    })
                except Exception:
                    continue

    return pd.DataFrame(resultados)

def main():
    """Função principal da aplicação Streamlit"""
    st.set_page_config(page_title="Classificador de Bulas", layout="wide")
    st.title("📋 Classificador Avançado de Bulas de Medicamentos")

    # Verificar pasta de exemplos
    if not os.path.exists(PASTA_EXEMPLOS):
        st.error(f"Pasta '{PASTA_EXEMPLOS}' não encontrada!")
        return

    # Carregar modelo
    tokenizer, model = carregar_modelo()

    # Listar arquivos PDF
    pdf_files = [f for f in os.listdir(PASTA_EXEMPLOS) if f.lower().endswith('.pdf')]
    if not pdf_files:
        st.warning(f"Nenhum arquivo PDF encontrado na pasta '{PASTA_EXEMPLOS}'")
        return

    # Interface
    st.sidebar.header("Configurações")
    arquivo_selecionado = st.sidebar.selectbox("Selecione um arquivo PDF", pdf_files)

    if arquivo_selecionado:
        pdf_path = os.path.join(PASTA_EXEMPLOS, arquivo_selecionado)

        with st.spinner(f'Processando {arquivo_selecionado}...'):
            texto = extrair_texto(pdf_path)

            if texto is None:
                st.error(f"Erro ao processar o arquivo: {arquivo_selecionado}")
                st.stop()

            df_resultados = processar_bula(texto, tokenizer, model, pdf_path)

        # Exibir resultados
        if not df_resultados.empty:
            st.subheader("📊 Estatísticas")
            col1, col2, col3, col4 = st.columns(4)
            col1.metric("Total de Sintomas", len(df_resultados))
            col2.metric("Sintomas Leves", len(df_resultados[df_resultados['Classificação'] == 'leve']))
            col3.metric("Sintomas Graves", len(df_resultados[df_resultados['Classificação'] == 'grave']))
            col4.metric("Sistemas Afetados", df_resultados['Sistema Afetado'].nunique())

            st.subheader("🔍 Resultados Detalhados")

            # Filtros avançados
            st.sidebar.subheader("Filtros Avançados")
            filtro_classificacao = st.sidebar.multiselect(
                "Classificação",
                options=['leve', 'grave'],
                default=['leve', 'grave']
            )

            sistemas_disponiveis = df_resultados['Sistema Afetado'].unique()
            filtro_sistema = st.sidebar.multiselect(
                "Sistema Afetado",
                options=sistemas_disponiveis,
                default=sistemas_disponiveis
            )

            secoes_disponiveis = df_resultados['Seção'].unique()
            filtro_secao = st.sidebar.multiselect(
                "Seção da Bula",
                options=secoes_disponiveis,
                default=secoes_disponiveis
            )

            df_filtrado = df_resultados[
                (df_resultados['Classificação'].isin(filtro_classificacao)) &
                (df_resultados['Sistema Afetado'].isin(filtro_sistema)) &
                (df_resultados['Seção'].isin(filtro_secao))
            ]

            st.dataframe(df_filtrado)

            # Visualizações adicionais
            st.subheader("📈 Análise Visual")
            tab1, tab2 = st.tabs(["Distribuição por Classificação", "Sistemas Afetados"])

            with tab1:
                st.bar_chart(df_resultados['Classificação'].value_counts())

            with tab2:
                st.bar_chart(df_resultados['Sistema Afetado'].value_counts())

            # Download
            csv = df_resultados.to_csv(index=False, sep=';').encode('utf-8')
            st.download_button(
                "⬇️ Baixar Resultados (CSV)",
                data=csv,
                file_name=f"resultados_{arquivo_selecionado.replace('.pdf', '.csv')}",
                mime="text/csv"
            )
        else:
            st.warning("Nenhum sintoma relevante encontrado neste arquivo.")

    # Rodapé
    st.sidebar.markdown("---")
    st.sidebar.subheader("Sobre")
    st.sidebar.info("""
    **Classificador de Bulas v2.0**\n
    - Analisa termos médicos complexos
    - Classifica por gravidade e sistema
    - Suporte a PDFs protegidos
    """)

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'streamlit'

# **Referências**

- **BERTimbau (BERT para Português)**  
  [Souza, F. et al. (2020). "BERTimbau: Pretrained BERT Models for Brazilian Portuguese"](https://arxiv.org/abs/2008.03129)

- **BERT: Pre-training of Deep Bidirectional Transformers**  
  [Devlin, J. et al. (2019). NAACL-HLT](https://arxiv.org/abs/1810.04805)

- **PyPDF2 Documentation**  
  [PyPDF2 Official Docs](https://pypdf2.readthedocs.io/)

- **Streamlit**  
  [Streamlit API Reference](https://docs.streamlit.io/)

- **Hugging Face Transformers**  
  [Wolf, T. et al. (2020). "Transformers: State-of-the-art Natural Language Processing"](https://arxiv.org/abs/1910.03771)

- **Clinical NLP em Português**  
  [Fonseca, E. et al. (2022). "Clinical Text Mining in Portuguese"](https://www.sciencedirect.com/science/article/pii/S2665963822000125)


---

In [ ]:
#@title **Avaliação**
GitHub = 10 #@param {type:"slider", min:0, max:10, step:1}

Implementacao_Model_Code = 7 #@param {type:"slider", min:0, max:10, step:1}

Aplicacao_Streamlit = 9 #@param {type:"slider", min:0, max:10, step:1}

Texto_Artigo  = 6 #@param {type:"slider", min:0, max:10, step:1}

Video = 7 #@param {type:"slider", min:0, max:10, step:1}

Geral = 7 #@param {type:"slider", min:0, max:10, step:1}








In [ ]:
#@title **Nota Final**

nota = 2*GitHub + 4*Implementacao_Model_Code + 2*Aplicacao_Streamlit + 1*Texto_Artigo + 1*Video

nota = nota / 10

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 7.9



,tia,nome,nota
0,1115665,ADRIANA FUJITA,7.9
1,1115677,DANIEL HENRIQUE,7.9
